# TODO

- EDA (correlation matrix) ; fouskakis 
- Feature extraction
- try classifiers 
- grid search -> find best 
- further research -> kaggle notebooks

# Imports

In [2]:
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

In [3]:
path = 'Data/'
RAND_STATE = 42

In [4]:
inputs = pd.read_csv(path + 'diabetes_012_health_indicators_BRFSS2015.csv')
df = inputs.copy()

# Preprocessing

In [5]:
df.columns

Index(['Diabetes_012', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker',
       'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')

In [6]:
df.isnull().sum().sum()

0

In [7]:
for col in df.columns:
    df[col] = df[col].astype(int)


In [8]:
labels = df[['Diabetes_012']].to_numpy().flatten()
labels

array([0, 0, 0, ..., 0, 0, 2])

In [9]:
df.drop(['Diabetes_012'], axis=1)

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,1,1,1,40,1,0,0,0,0,1,...,1,0,5,18,15,1,0,9,4,3
1,0,0,0,25,1,0,0,1,0,0,...,0,1,3,0,0,0,0,7,6,1
2,1,1,1,28,0,0,0,0,1,0,...,1,1,5,30,30,1,0,9,4,8
3,1,0,1,27,0,0,0,1,1,1,...,1,0,2,0,0,0,0,11,3,6
4,1,1,1,24,0,0,0,1,1,1,...,1,0,2,3,0,0,0,11,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253675,1,1,1,45,0,0,0,0,1,1,...,1,0,3,0,5,0,1,5,6,7
253676,1,1,1,18,0,0,0,0,0,0,...,1,0,4,0,0,1,0,11,2,4
253677,0,0,1,28,0,0,0,1,1,0,...,1,0,1,0,0,0,0,2,5,2
253678,1,0,1,23,0,0,0,0,1,1,...,1,0,3,0,0,0,1,7,5,1


In [10]:
_ , class_samples = np.unique(labels, return_counts=True)
class_samples/sum(class_samples)

array([0.8424117 , 0.01825528, 0.13933302])

0: Υγιείς
1: Προδιαβητικοί
2: Διαβητικοί

In [12]:
len(df[df['Diabetes_012']==0]),  len(df[df['Diabetes_012']==1]), len(df[df['Diabetes_012']==2])

(213703, 4631, 35346)

## Random Forest Classifier

In [9]:
train_data, test_data, train_labels, test_labels = train_test_split(df, labels, 
                                                                    test_size = 0.25,
                                                                    random_state = RAND_STATE,
                                                                    stratify=labels)
# Stratify = y creates train and test data in a way that every label is represented with the same percentage on both train and test subsets


In [10]:
rf = RandomForestClassifier(random_state=RAND_STATE,
                                       class_weight='balanced')

In [11]:
rf.fit(train_data, train_labels)

RandomForestClassifier(class_weight='balanced', random_state=42)

In [12]:
rf.score(test_data, test_labels)

1.0

In [20]:
# print(f'train labels classes:{np.unique(train_labels, return_counts=True)}')
# print(f'test labels classes:{np.unique(test_labels, return_counts=True)}')
_ , class_samples = np.unique(train_labels, return_counts=True)
class_samples/sum(class_samples)

array([0.84241039, 0.01825397, 0.13933565])

In [14]:
print(classification_report(test_labels, rf.predict(test_data)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     53426
           1       1.00      1.00      1.00      1158
           2       1.00      1.00      1.00      8836

    accuracy                           1.00     63420
   macro avg       1.00      1.00      1.00     63420
weighted avg       1.00      1.00      1.00     63420



In [15]:
test_labels

array([0, 0, 0, ..., 0, 0, 0])

# Logistic Regression

In [17]:
lr = LogisticRegression(random_state = RAND_STATE, class_weight = 'balanced')
lr.fit(train_data, train_labels)
lr.score(test_data, test_labels)

C:\Users\milio\anaconda3\envs\edemm\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9999842321034375

In [18]:
print(classification_report(test_labels, lr.predict(test_data)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     53426
           1       1.00      1.00      1.00      1158
           2       1.00      1.00      1.00      8836

    accuracy                           1.00     63420
   macro avg       1.00      1.00      1.00     63420
weighted avg       1.00      1.00      1.00     63420

